# Dead toggle extraction from all versions

### Identify and match toggles found in all files with config files

In [1]:
import os
import glob
import re
import pandas as pd

In [10]:
system_root = '/Users/govardhanrathamsetty/Desktop/ToggleSmell-Chromium/All versions/comp_ver'
#system_root = '/home/taj/Documents/ArchPrediction/ProcessedVersions'
ch_version = '45.0.2404.0'

### Finding Toggle Variables and Identifying if They are Dead

In [40]:
def find_dead(toggles_list_form_config):
            all_cc_files = glob.glob(f'{system_root}/chromium {ch_version}/**/*.cc', recursive=True)
            
            container1=[]
            container2= []
            container3=[]
            container4=[]

            for cc_file in all_cc_files:
                if 'switch' not in cc_file:
                    if 'feature' not in cc_file:
                        with open(cc_file, 'rb') as file:  
                            try:
                                content = file.read().decode('utf-8')  
                                container1.append(content)
                            except UnicodeDecodeError:
                                pass
    
            reg = [r'switches\::k[A-Z].*?\)', r'\s*if\s*\(k[A-Z].*\)', r'\s*\(k[A-Z]', r'\::k[A-Z].*']

            for file_content in container1:
                for r in reg:
                    matches = re.findall(r, file_content)
                    container2.extend(matches)

            dt_list = [re.findall(r'\b[k]\w*\b', line) for line in container2]

            # TODO: Need to get back to this line because the cut-off threshold of 10 is not fully determined
            dt_list = [j for i in dt_list for j in i if len(j) > 10]

            for dt in list(set(dt_list)):
                if len(dt) > 10 and dt not in list(set(toggles_list_form_config)):
                    container3.append(dt)

            print( (len(list(set(dt_list))), len(list(set(container3)))) )
            print( list(set(container3)) )

### Extract Dead Toggles

In [41]:
def extract_dead_toggles():
    switch = glob.glob(f'{system_root}/chromium {ch_version}/**/*_switches.cc', recursive=True)
    feature = glob.glob(f'{system_root}/chromium {ch_version}/**/*.cc', recursive=True)

    config_files = switch + feature

    toggle_list = []

    for conf_file in config_files:
           with open(conf_file, 'r') as file:
                    file_content = file.read()
                    toggle_list.append(file_content)

    toggle_list = list(filter(None, toggle_list))
    toggle_patterns = [r'const char k[A-Z].*', r'\::Feature k[A-Z].*', r'\s*k[A-Z].*']
    found_toggles = []
    toggles = []

    for toggle in toggle_list:
        for pattern in toggle_patterns:
            matches = re.findall(pattern, toggle)
            found_toggles.extend(matches)
    found_toggles = list(filter(None, found_toggles))

    for k_toggles in found_toggles:
        toggles.extend(re.findall(r'\b[k]\w*\b',k_toggles))

    return find_dead(toggles)

In [42]:
extract_dead_toggles()

(6970, 6211)
['kIconProperty', 'kInstallDate', 'kScrollbarHorizontalTrack', 'kObsoleteOriginDirectory', 'kStabilityExecutionPhase', 'kDataReductionPassThroughHeader', 'kRequestShutdownMethod', 'kOmniboxKeyword', 'kGeolocation', 'kDistroSuppressFirstRunBubble', 'kContentSettingsPopupsPatternPairs', 'kClearSiteDataOnExit', 'kSessionLengthLimit', 'kBaseDescriptor', 'kChromeUISupervisedUserPassphrasePageHost', 'kActiveInputNodeChanged', 'kSyncedNotificationAppInfoFieldNumber', 'kSessionManagerStoreDeviceLocalAccountPolicy', 'kPanelMinWidth', 'kDeviceLoginScreenDefaultLargeCursorEnabled', 'kManagedJavaScriptAllowedForUrls', 'kTxPowerProperty', 'kRegPathClientState', 'kChromeUITracingHost', 'kCookiesSessionOnlyForUrls', 'kUnknownVideoCodecProfile', 'kPRegFileName', 'kWebConnectable', 'kCursorNorthEastSouthWestResize', 'kAdmErrNone', 'kGuestUserName', 'kAudioDspInterface', 'kDisconnectProfile', 'kSimpleEntryFileCount', 'kGoogTypingNoiseDetection', 'kPreviewInitiatorRoutingId', 'kHotwordFieldT

## DEAD-TOGGLE-PATH-MAPPING

In [24]:
ch_version = '90.0.4390.0'

In [25]:
def dead_toggle_mapping():
    cc_files = glob.glob(f'{system_root}/chromium {ch_version}/**/*.cc', recursive=True)

    code_path_dic = {}

    for cc_file in cc_files:
           with open(cc_file, 'r') as file:
                    file_contents = file.read()
                    code_path_dic[cc_file] = file_contents

    code_path_dic = {key: value for key, value in emp_dic.items() if value is not None and value != ''}
    
    code_lines=[]
    path_tog_match =[]
    toggle_pattern = [r'switches\::k[A-Z].*?\)', r'\s*if\s*\(k[A-Z].*\)', r'\s*\(k[A-Z]', r'\::k[A-Z].*']
    for path, code in list(code_path_dic.items())[:50]:
        parts = path.split('/')
        file_name = parts[-1]
        version = '/'.join(parts[7:8])
        code = code.split('\n')
        code_lines.append(code)
        for code_line in code_lines:
            for line in code_line: 
                for reg in toggle_pattern:
                    matches = re.findall(reg, line)
                    matches = [re.findall(r'\s*[k].*',mat) for mat in matches]
                    for dead in matches:
                        if dead not in final_toggle_list:
                            path_tog_match.append((dead[0], path, file_name, version))
    mapping_df = pd.DataFrame(path_tog_match, columns =['Toggle', 'Path', 'File_name', 'Version'])
    return mapping_df

dead_toggle_mapping()

,Toggle,Path,File_name,Version
0,kUpperLeft_Corner)));,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,benchmarking_canvas.cc,chromium 90.0.4390.0
1,kUpperRight_Corner)));,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,benchmarking_canvas.cc,chromium 90.0.4390.0
2,kLowerRight_Corner)));,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,benchmarking_canvas.cc,chromium 90.0.4390.0
3,kLowerLeft_Corner)));,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,benchmarking_canvas.cc,chromium 90.0.4390.0
4,"kAlphaUnchanged_Flag,",/Users/govardhanrathamsetty/Desktop/ToggleSmel...,benchmarking_canvas.cc,chromium 90.0.4390.0
...,...,...,...,...
3054,kPiDouble / 2);,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,audio_silence_detector.cc,chromium 90.0.4390.0
3055,"kPiDouble / 2,",/Users/govardhanrathamsetty/Desktop/ToggleSmel...,audio_silence_detector.cc,chromium 90.0.4390.0
3056,kPiDouble);,/Users/govardhanrathamsetty/Desktop/ToggleSmel...,audio_silence_detector.cc,chromium 90.0.4390.0
3057,"kPiDouble,",/Users/govardhanrathamsetty/Desktop/ToggleSmel...,audio_silence_detector.cc,chromium 90.0.4390.0


ABOVE TOGGLE VARIABLES NEED TO BE CLEANED PROPERLY.